# Weeks 4 & 5: Formulating a Framework for Opportunity Zone Investment in Washington, D.C.

## Introduction

For debt and equity investors, as well as government support programs and city planning, opportunity zones have been a tax benefit strategy of intense interest. Understanding the socio-economic implications of certain investments in under-served areas has been exacerbated in 2020 by the Coronavirus pandemic.

This analysis builds off of recent work establishing one-way, time delayed correlation between particular business category entrants to a zip code and the gentrification of a neighborhood, as best defined by the change in Streetscore and migration of college educated individuals [1]. Understanding the knock-on effects of gentrification, it is the author's intent to create a dialogue focused around the profile of neighborhoods primed for gentrification, to better identify the sorts of investments that might "raise all boats". Further research by the same authors points to complimentary roles census data can have when paired with private sector resources, and so census data is pulled into the visual framework to build out a more robust profile of the neighborhoods in Washington, D.C.

Principle to the Yelp study findings were the predictive power of increases in vegetarian restaurants, cafes and wine bars to positive Streetscore changes, where Streetscore is a metric generated by AI techniques to quantify the percieved safety of a neighborhood [3]. Examining these factors in Washington, D.C. the author hopes to further this area of research by leveraging Foursquare's public API to map out density clusters of the highlighted venues. In this way, the likelihood for expanding gentrification can be flagged, and the proper investments can be screened to create jobs that cover the full vertical of tax brackets, i.e. higher tech manufacturing.

## Data

Foursquare data will be pulled over the public API to collect venue category and location data for the Washington, D.C. area. Specifically, vegetarian restaurants, cafes and wine bars will be selected due to their demonstrated correlation with the gentrification of neighborhoods.

Government census data will be pulled to map empoyment, unemployment and opportunity zone qualifying census tract data onto the Washington, D.C. area. To best visualize this data, different technqiues will be leveraged from the folium library. Census tract data will be visualized with the plug-in, Foursquare data will be mapped with the clustering plug-in, and circles visualizing the five-year employment/unemployment product change be displayed concurrently.

## Methodology

First, all of the necessary libraries and packages are downloaded.

In [1]:
import requests
import random

#!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim

from IPython.display import Image 
from IPython.core.display import HTML 
    
from pandas import json_normalize

#!conda install -c conda-forge folium=0.5.0 --yes
import folium

In [2]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import json
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans

Then the process of data wrangling begins. We use the API for Census.gov to pull 2013 and 2018 data on employment and unemployment by DC sector. The sectors are drawn rather abstractly, and so best guess coordinates are assigned to each of the sectors. To simplify the arguement, a ratio of the two years are computed and then multiplied to assign an equal weight to employment gains as are assigned unemployment decreases.

In [3]:
url = 'https://api.census.gov/data/2013/acs/acs1/pums?tabulate=weight(PWGTP)&col+ESR&row+ucgid&ucgid=0400000US11,7950000US1100101,7950000US1100102,7950000US1100103,7950000US1100104,7950000US1100105'
CensusList13 = requests.get(url).json()
CensusList13.pop(0)
CensusList13.pop(5)
df_cl13 = pd.DataFrame(CensusList13)
df_cl13 = df_cl13[[0, 5]]
df_cl13.rename(columns={0: "Unemployed", 5: "Employed"}, inplace = True)
dc_loc = {'Latitude': [38.897851, 38.890774, 38.938858, 38.969828, 38.928710], 'Longitude': [-77.031744, -76.954496, -77.002905, -77.033804, -77.079466]}
df_dc_loc = pd.DataFrame(dc_loc)
c13 = pd.concat([df_cl13, df_dc_loc], axis = 1, sort = False)

In [4]:
url = 'https://api.census.gov/data/2018/acs/acs1/pums?tabulate=weight(PWGTP)&col+ESR&row+ucgid&ucgid=0400000US11,7950000US1100101,7950000US1100102,7950000US1100103,7950000US1100104,7950000US1100105'
CensusList18 = requests.get(url).json()
CensusList18.pop(0)
CensusList18.pop(5)
df_cl18 = pd.DataFrame(CensusList18)
df_cl18 = df_cl18[[0, 5]]
df_cl18.rename(columns={0: "Unemployed", 5: "Employed"}, inplace = True)
dc_loc = {'Latitude': [38.897851, 38.890774, 38.938858, 38.969828, 38.928710], 'Longitude': [-77.031744, -76.954496, -77.002905, -77.033804, -77.079466]}
df_dc_loc = pd.DataFrame(dc_loc)
c18 = pd.concat([df_cl18, df_dc_loc], axis = 1, sort = False)

In [5]:
Unemployment_Loss = c13['Unemployed'].div(c18['Unemployed'])
Employment_Gain = c18['Employed'].div(c13['Employed'])
employment_impact = pd.DataFrame(columns = ['5 Year Employment Impact', 'Latitude', 'Longitude'])
employment_impact['5 Year Employment Impact'] = np.multiply(Employment_Gain, Unemployment_Loss)
employment_impact[['Latitude', 'Longitude']] = c18[['Latitude', 'Longitude']]

A procedure is then created to optimize the quantity of data being pulled from Foursquare. First a list of coordinates for neighborhoods is created, extracted as a GeoJSON file and flattened down into a one dimensional data frame. Our last step in preparation for the Foursquare data pull is to define a function where get requests for each neighborhood cluster coordinate are looped over.

In [6]:
# Creating a dataframe of neighborhoods and their respective coordinates in D.C. to iterate get requests off of.

url = 'https://maps2.dcgis.dc.gov/dcgis/rest/services/DCGIS_DATA/Administrative_Other_Boundaries_WebMercator/MapServer/17/query?where=1%3D1&outFields=NBH_NAMES&outSR=4326&f=json'
results = requests.get(url).json()  
locations = results['features']

# Flattening JSON file
neighborhood_locations = json_normalize(locations)

# Filter columns
filter_columns = ['attributes.NBH_NAMES', 'geometry.rings']
neighborhood_locations = neighborhood_locations.loc[:, filter_columns]

# Clean columns
neighborhood_locations.columns = [col.split(".")[-1] for col in neighborhood_locations.columns]

# define the dataframe columns
column_names = ['Neighborhoods', 'Location'] 

# instantiate the dataframe and correct columns
neighborhoods = pd.DataFrame(columns=column_names)

neighborhoods['Neighborhoods'] = neighborhood_locations['NBH_NAMES']
neighborhoods['Location'] = neighborhood_locations['rings'].str[0].str[0]
neighborhoods['Latitude'] = neighborhoods['Location'].str[1]
neighborhoods['Longitude'] = neighborhoods['Location'].str[0]
neighborhoods.drop(columns = 'Location', inplace = True)


In [7]:
# Base Foursquare Credentials

CLIENT_ID = 'E1DDUUEYLNNLJ54YLM5VUUXDMD5SHVFJVJS2OYY1FNHP1GHM'
CLIENT_SECRET = 'TNT1LN3QRSRPFQBUVWD4YDEBHDWJ5PCXAIQFHQWFQL4BQ5XG'
VERSION = '20180604'
LONGITUDE = -77.035864
LATITUDE = 38.901324
RADIUS = 1000
LIMIT = 100
CATEGORYID = '4bf58dd8d48988d16d941735,4bf58dd8d48988d1d3941735,4bf58dd8d48988d123941735'


In [8]:
def getNearbyVenues(names, latitudes, longitudes):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}&categoryId={}'.format(
            CLIENT_ID,
            CLIENT_SECRET,
            VERSION,
            lat,
            lng,
            RADIUS,
            LIMIT,
            CATEGORYID)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhoods', 
                  'Neighborhoods Latitude', 
                  'Neighborhoods Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [9]:
# type your answer here

DC_venues = getNearbyVenues(names=neighborhoods['Neighborhoods'],
                                   latitudes=neighborhoods['Latitude'],
                                   longitudes=neighborhoods['Longitude']
                                  )

The final step in data wrangling will be to ensure that venue categories are truly filtered to the desired subset, and to assign true values to every census tract that is eligible for opportunity zone investing.

In [10]:
FilterList = ['Café', 'Wine Bar', 'Vegetarian / Vegan Restaurant']

from sklearn.preprocessing import StandardScaler
DC_filtered = DC_venues[DC_venues['Venue Category'].isin(FilterList)].reset_index(drop=True)
DC_unique = DC_filtered.drop_duplicates(subset = ['Venue']).reset_index(drop=True)


In [11]:
oz = ['006400', '007304', '003400',  '010900', '010400', '007401', '007503', '007604', '007804', '007808', '009602', '009603', '007601', '007603', '007709', '007803', '007806', '007407', '009601', '006804' , '002101', '008904' , '009102', '009204' , '010300']

!wget --quiet https://opendata.arcgis.com/datasets/6969dd63c5cb4d6aa32f15effb8311f3_8.geojson -O 6969dd63c5cb4d6aa32f15effb8311f3_8.geojson
DC_CensusTracts = r'6969dd63c5cb4d6aa32f15effb8311f3_8.geojson' # geojson file    

url = 'https://opendata.arcgis.com/datasets/6969dd63c5cb4d6aa32f15effb8311f3_8.geojson'
results = requests.get(url).json()
tract_features = results['features']
df_tract = json_normalize(tract_features)
tract = df_tract['properties.TRACT']
df_t = pd.DataFrame(data = tract)
df_t['OZ'] = df_t.isin(oz)
df_t['OZ'] = df_t['OZ'].astype(int)
OZ_Tracts_Recoded = df_t.rename(columns = {'properties.TRACT':'Census Tracts', 'OZ':'Opportunity Zone?'})


The folium library will be used generate clusters, chloropleth map census tracts and assign a color scheme to opportunity zone tracts. Additionally, markers will be created to represent the product of employment gains and unemployment decreases with radius for magnitude, and color for net increase vs. decrease.

In [12]:
from folium import plugins

LATITUDE = 38.901324
LONGITUDE = -77.035864
map_clusters = folium.Map(location=[LATITUDE, LONGITUDE], tiles = "Stamen Terrain", zoom_start=12)

def condcolor(i):
    if i > 1:
        return 'black'
    else:
        return 'red'

clusters = plugins.MarkerCluster().add_to(map_clusters)

map_clusters.choropleth(
    geo_data=DC_CensusTracts,
    data=OZ_Tracts_Recoded,
    columns=['Census Tracts', 'Opportunity Zone?'],
    key_on='feature.properties.TRACT',
    fill_color='GnBu', 
    fill_opacity=0.4, 
    line_opacity=0.2,
    legend_name='Opportunity Zone Tracts'
)

markers_colors = []
for name, cat, lat, lon in zip(DC_unique['Venue'], DC_unique['Venue Category'], DC_unique['Venue Latitude'], DC_unique['Venue Longitude']):
    label = folium.Popup('Name: ' + str(name) + ', Category: ' + str(cat), parse_html=True)
    folium.Marker(
        [lat, lon],
        popup=label,
        ).add_to(clusters)
    
zone_rings = []
for value, lat, lon in zip(employment_impact['5 Year Employment Impact'], employment_impact['Latitude'], employment_impact['Longitude']):
    label = folium.Popup('Employment Impact: ' + str(value), parse_html=True)
    folium.Circle(
        [lat, lon],
        radius = 2000*value,
        popup = label,
        color = condcolor(value),
        fill=True,
        fill_color = condcolor(value),
        fill_opacity = .1
        ).add_to(map_clusters)
       
map_clusters

## Results

After visualizing clusters and observing relative placement amongst employment figures and opportunity zones, it becomes clear that three options exist: (A) opportunity zones in areas primed for gentrification experiencing rising unemployment, (B) opportunity zones in areas experiencing an economic boom primed for gentrification, (C) and opportunity zones in areas experiencing growing economic activity away from gentrification primers.

Of the three, (A) is an area of special concern. Prolonged and growing unemployment paired with investments that raise property taxes run a risk of displacing vulnerable population. Differently, the northern sector of DC has a large, immediately available workforce, and a dynamic of diverse living arrangements that allow for industries that employ the full vertical of skill.

## Discussion

From here, further research needs to be done into the historic employers for the neighborhoods around the northern opportunity zone. Disaggregation of typical jobs held into skills, both hard and soft, will allow for the proper mapping of skills available within the labor pool for industrial or commercial investment. Of the strongest correlation discovered in prior research, the venues marked are most likely to preceed neighborhood improvements in the form of high rent high rises. This would obviously be a disastrous outcome for a community suffering from unemployment and low job availability.

Within the census data built into the chloropleth map are poverty rates. Ultimately, we would want to be able to pull foursquare data in five year increments prior to employment and poverty figures to test out for correlation between the arrival of such venue categories and their delayed microeconomic impact. Furthermore, mapping out the density of different skills within the census tracts, or at the minimum D.C. sectors would allow for a more developed, prescriptive framework.

## Conclusion

In at least the D.C. market, the visual findings from this report allude to the possiblity of a framework built out from census and private entity data. To move past this descriptive analysis, future research will aim to collect the same data for several years prior.

While this report leverages some research created from D.C. data, the rest of the research is extrapolated from the similar metropolitan markets and it would make sense to recreate some of the findings specifically for D.C.

It's the authors opinion that policy derived from the sort of framework alluded to in this report is critical to correct for frequent pattern of displacement of underserved populations.

## References

[1] Glaeser, E.L., Kim, H., Luca, M. "MEAUSURING GENTRIFICATION: USING YELP DATA TO QUANTIFY NEIGHBORHOOD CHANGE"
[2] Glaeser, E.L., Kim, H., Luca, M. "NOWCASTING THE LOCAL ECONOMY:
USING YELP DATA TO MEASURE ECONOMIC ACTIVITY"
[3] Naik, N., S.D. Kominers, R. Raskar, E.L. Glaeser, "Computer vision uncovers predictors of physical urban change"